In [1]:
import json
from urllib.parse import urlencode
from urllib.request import urlretrieve
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import requests
from lxml import html
import re
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

idx = pd.IndexSlice

In [2]:
def get_nba_data(endpt, params, return_url=False):

    ## endpt: https://github.com/seemethere/nba_py/wiki/stats.nba.com-Endpoint-Documentation
    ## params: dictionary of parameters: i.e., {'LeagueID':'00'}

    useragent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""
    dataurl = "\"" + "http://stats.nba.com/stats/" + endpt + "?" + urlencode(params) + "\""
    
    # for debugging: just return the url
    if return_url:
        return(dataurl)
    
    jsonstr = !wget -q -O - --user-agent={useragent} {dataurl}
    
    data = json.loads(jsonstr[0])
    
    h = data['resultSets'][0]['headers']
    d = data['resultSets'][0]['rowSet']
    
    return(pd.DataFrame(d, columns=h))

def get_params(url):
    endpt = url.split('stats/')[1].split('?')[0]
    params = {}
    paramstring = url.split('stats/')[1].split('?')[1]

    for substring in paramstring.split("&"):
        variable = substring.split('=')[0]
        value = substring.split('=')[1]
        params.update({variable:value})
    return([endpt,params])
user_agent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""

headers = {'User-Agent':user_agent}

def SportsRefCBBRowMaker(player_url,table_list,player_name,correct_year,player_id="0"):
    
    page = requests.get(player_url,headers=headers)
    tree = html.fromstring(page.content)
    
    if '404' in tree.xpath('//title')[0].text:
        return('404')
    else:
        year = tree.xpath('//table/tbody/tr')[-1].items()[0][-1][-4:]
        if(year!=correct_year):
            return(year)
        
        for i,table_name in enumerate(table_list):
            path = '//div[@id="'+table_name+'"]'
            try:
                treeStr = str(tree.xpath(path)[0].getchildren()[-1])[8:-4]
            except:
                continue
            #This is tricky and comments might be nice here. 

            tree2 = html.fromstring(treeStr)

            valuesHTML = tree2.xpath('//table/tfoot/tr')[-1].getchildren()[1:]
            ## This gives career in particular 
            conf = tree2.xpath('//table/tbody/tr/td')[1].getchildren()[0].text
            colNamesHTML = tree2.xpath('//table/thead/tr/th')
            colNames = ["PLAYER_ID","PLAYER_NAME"]+["CONF"] + [col.text for col in colNamesHTML][1:]
            values = [player_id,player_name] + [conf] + [val.text for val in valuesHTML]
            if(i==0):
                row = pd.DataFrame(pd.Series(values,index=colNames)\
                    .dropna()).transpose()
            else:
                row = pd.merge(row,pd.DataFrame(pd.Series(values,index=colNames)\
                    .dropna()).transpose())
        return(row)
    
def sportsRefCBBTableMaker(player_df, table_list):
    base_url = 'https://www.sports-reference.com/cbb/players/'
    columns_df_url = "https://www.sports-reference.com/cbb/players/malcolm-brogdon-1.html"
    columns_df = SportsRefCBBRowMaker(columns_df_url,table_list
                                      ,"Malcolm Brogdon(2016)",'2016')
    CBB_df = pd.DataFrame(columns=columns_df.columns)
    for index,row in player_df.iterrows():
        player_name = row['PLAYER_NAME']
        first = re.sub('[^a-zA-Z]+', '',player_name.split()[0].lower())
        last = "".join(player_name.split()[1:])[0:-6].lower().strip(".")
        year = player_name.split()[1][-5:-1]
        name_url = base_url + first + "-" + last + "-"

        i = 1
        stopping_condition=False
        while(stopping_condition==False):
            data_url = name_url + str(i) + ".html"
#            print(data_url)
            row = SportsRefCBBRowMaker(data_url,table_list,player_name,year,player_id=index)
            if isinstance(row, pd.DataFrame):
                stopping_condition=True
                continue
            if(row=='404'):
                stopping_condition=True
                row = None
                if(last[-2:]=="jr"):
                    last = last[:-2]
                    name_url = base_url + first + "-" + last + "-"
                    i=1
                    stopping_condition = False 
            i+=1
        CBB_df = pd.concat([CBB_df,row])
    return(CBB_df)

In [3]:
main_df = pickle.load(open('NBA_2014_18_Adv_100_Score_Def.pkl','rb'))
Player_ID_Dict = pickle.load(open('NBAPlayerDict.pkl','rb'))

In [4]:
main_df.head(15)

PLAYER_NAME   AGE  AST_PCT  AST_RATIO  \
PLAYER_ID SEASON SEASON_TYPE                                                 
101106    2014   Regular Season     Andrew Bogut  29.0    0.087       18.3   
          2015   Playoffs           Andrew Bogut  30.0    0.121       25.2   
                 Regular Season     Andrew Bogut  30.0    0.149       27.3   
          2016   Playoffs           Andrew Bogut  31.0    0.127       22.7   
                 Regular Season     Andrew Bogut  31.0    0.145       29.7   
          2017   Regular Season     Andrew Bogut  32.0    0.133       27.6   
          2018   Regular Season     Andrew Bogut  33.0    0.099       25.1   
101107    2014   Regular Season  Marvin Williams  28.0    0.075       11.3   
          2015   Regular Season  Marvin Williams  29.0    0.078       14.4   
          2016   Playoffs        Marvin Williams  30.0    0.043        9.9   
                 Regular Season  Marvin Williams  30.0    0.074       11.2   
          2017   Regular Season  Marvin Williams  31.0    0.072       11.3   
          2018   Regular Season  Marvin Williams  31.0    0.069       12.3   
101108    2014   Playoffs             Chris Paul  29.0    0.427       34.1   
                 Regular Season       Chris Paul  29.0    0.459       36.2   

                                 AST_TO  DEF_RATING  DREB_PCT  EFG_PCT  GP  \
PLAYER_ID SEASON SEASON_TYPE                                                 
101106    2014   Regular Season    1.15        98.8     0.293    0.627  67   
          2015   Playoffs          1.38        98.9     0.275    0.560  19   
                 Regular Season    1.70        95.2     0.262    0.563  67   
          2016   Playoffs          1.58        95.4     0.203    0.623  22   
                 Regular Season    1.95        97.2     0.252    0.629  70   
          2017   Regular Season    1.14        97.4     0.320    0.469  27   
          2018   Regular Season    0.79        99.7     0.251    0.680  23   
101107    2014   Regular Season    1.47       110.4     0.179    0.519  66   
          2015   Regular Season    1.67       102.2     0.183    0.520  78   
          2016   Playoffs          3.00       109.5     0.225    0.333   7   
                 Regular Season    1.77       102.6     0.187    0.554  81   
          2017   Regular Season    1.77       106.4     0.202    0.510  76   
          2018   Regular Season    1.45       105.5     0.155    0.570  78   
101108    2014   Playoffs          3.44       107.6     0.114    0.548  13   
                 Regular Season    4.57       100.1     0.114    0.511  62   

                                  MIN     ...      DEF_FREQ_3Pt  \
PLAYER_ID SEASON SEASON_TYPE              ...                     
101106    2014   Regular Season  26.4     ...             0.064   
          2015   Playoffs        23.1     ...             0.055   
                 Regular Season  23.6     ...             0.065   
          2016   Playoffs        16.6     ...             0.131   
                 Regular Season  20.7     ...             0.082   
          2017   Regular Season  21.6     ...             0.084   
          2018   Regular Season   9.4     ...             0.177   
101107    2014   Regular Season  25.4     ...             0.263   
          2015   Regular Season  26.1     ...             0.245   
          2016   Playoffs        32.5     ...             0.367   
                 Regular Season  28.9     ...             0.293   
          2017   Regular Season  30.2     ...             0.362   
          2018   Regular Season  25.7     ...             0.331   
101108    2014   Playoffs        36.3     ...             0.383   
                 Regular Season  35.0     ...             0.352   

                                 DEF_FREQ_Gt15  DEF_FREQ_Lt10  DEF_FREQ_Lt6  \
PLAYER_ID SEASON SEASON_TYPE                                                  
101106    2014   Regular Season          0.267          0.628         0.483   
      

In [ ]:
draft_history = get_nba_data('drafthistory',{'LeagueID':'00'})

In [ ]:
draft_history['SEASON']=pd.Series(draft_history.SEASON,dtype='int64')
draft_history = draft_history.query('SEASON>=2010')
draft_history = draft_history.query("ORGANIZATION_TYPE == 'College/University'")

draft_history['PLAYER_ID']=pd.Series(draft_history['PERSON_ID'],dtype=str)

draft_history = draft_history.set_index('PLAYER_ID')
#draft_history = draft_history.drop('PERSON_ID',axis=1)

draft_history.head()

In [ ]:
draft_history['ORGANIZATION'][draft_history.ORGANIZATION=='California-Los Angeles']='ucla'
draft_history['ORGANIZATION']= draft_history.ORGANIZATION.str.lower()
draft_history.query('SEASON==2017').ORGANIZATION.unique()

In [ ]:
qualified_player_list = main_df.loc[idx[:,:,'Regular Season'],:].reset_index().query('SEASON_MIN>700').PLAYER_ID.unique()

In [ ]:
combine_columns = ['PLAYER_NAME','DRAFT_YEAR','POSITION','STANDING_VERTICAL_LEAP',
                   'MAX_VERTICAL_LEAP','LANE_AGILITY_TIME',
                   'MODIFIED_LANE_AGILITY_TIME','THREE_QUARTER_SPRINT',
                   'BENCH_PRESS','HEIGHT_WO_SHOES','WEIGHT','WINGSPAN',
                   'STANDING_REACH','BODY_FAT_PCT','HAND_LENGTH','HAND_WIDTH'                  
                  ]
params = {'LeagueID':'00'}
full_combine_df = pd.DataFrame(columns = combine_columns)
for i in range(8,19):
    year="20"+format(i,'02d')+"-"+format(i+1,'02d')
    params.update({"SeasonYear":year})
    df1 = get_nba_data(endpt="draftcombinedrillresults",params=params)
    df2 = get_nba_data(endpt="draftcombineplayeranthro",params=params)
    dfMerged = pd.merge(df1,df2)
    dfMerged['PLAYER_ID']=pd.Series(dfMerged.PLAYER_ID,dtype=str)
    if(i>13):
        dfMerged['PLAYER_NAME'] = dfMerged['PLAYER_NAME']
        dfMerged['PLAYER_ID']=(dfMerged['PLAYER_NAME']+"("+year[0:4]+")").map(Player_ID_Dict)
        dfMerged['PLAYER_ID']=pd.Series(dfMerged['PLAYER_ID'],dtype=str).str.split(".").str.get(0)

    dfMerged['DRAFT_YEAR'] = int(year[0:4])    
    dfMerged = dfMerged[dfMerged['PLAYER_ID']!='nan']

    dfMerged = dfMerged.set_index('PLAYER_ID')[combine_columns]
        
    full_combine_df = pd.concat([full_combine_df,dfMerged])
full_combine_df.fillna(value=np.nan, inplace=True)
full_combine_df = full_combine_df.drop(['MODIFIED_LANE_AGILITY_TIME','BENCH_PRESS'],axis=1)

In [ ]:
full_combine_df.tail()

In [ ]:
qualified_player_list = main_df.loc[idx[:,:,'Regular Season'],:].reset_index().query('SEASON_MIN>700').PLAYER_ID.unique()
new_player_list=np.intersect1d(qualified_player_list,np.array(full_combine_df.index))

In [ ]:
qual_combine_df  = full_combine_df.loc[new_player_list,:]
len(qual_combine_df)

We have 200 players drafted after 2010 who have full combien results and  played enough minutes in at least one season to qualify. 
Not sure if this is going to be enough but lets work off it. Now 189 for players who were drafted.

Lets first pull based on college to minimize excessive bball ref requests.

Saves 50 out of 200 requests?
Lets just move on 

In [ ]:
rowAdvanced = SportsRefCBBRowMaker('https://www.sports-reference.com/cbb/players/malcolm-brogdon-1.html',
                         ['all_players_per_poss','all_players_advanced'],'Malcolm Brogdon(2016)','2016')

In [ ]:
qual_combine_df['PLAYER_NAME'] = qual_combine_df['PLAYER_NAME'] + "(" +\
            pd.Series(qual_combine_df.DRAFT_YEAR,dtype=str).str[:4] + ")"
qual_combine_df.PLAYER_NAME

In [ ]:
college_stats_df=sportsRefCBBTableMaker(qual_combine_df,['all_players_per_min','all_players_advanced'])

No per 100 possesion from 2010 is interseting. Lets go back to draft and see what to do, Ok lets just do per 40 minutes instead

In [ ]:
#college_stats_df = college_stats_df.set_index('PLAYER_ID')

In [ ]:
college_draft_prospects = pd.merge(college_stats_df,full_combine_df,left_index=True,right_index=True)

In [ ]:
df = college_draft_prospects.apply(pd.to_numeric,errors='ignore')
df = full_combine_df
df.POSITION.unique()

Lets reposition to Guard, Wing, Forward, Big

Lets fill in missing data here. Lets do it by average of position if we can figure that out.

In [ ]:
df['POSITION_SIMPLE']=df.POSITION
df.POSITION_SIMPLE[df.POSITION_SIMPLE.str.contains('C')]='Big'
df.POSITION_SIMPLE[df.POSITION_SIMPLE.str.contains('PF')]='Forward'
df.POSITION_SIMPLE[df.POSITION_SIMPLE.str.contains('SF')]='Wing'
df.POSITION_SIMPLE[df.POSITION_SIMPLE.str.contains('G')]='Guard'
df.loc[:,['POSITION','POSITION_SIMPLE']]

Start working on handling missing values.

Fix combine data based on new position averages.
Instead maybe we do a linear model of weight, and height?

In [ ]:
df.count()
df.loc[:,['STANDING_VERTICAL_LEAP','MAX_VERTICAL_LEAP','LANE_AGILITY_TIME',\
   'THREE_QUARTER_SPRINT','HAND_LENGTH','HAND_WIDTH','WEIGHT','HEIGHT_WO_SHOES','POSITION_SIMPLE']]\
    .groupby('POSITION_SIMPLE').count()


In [ ]:
df.columns

Lets take a little step back and focus on filling out data a little more for those who missed combine.

We'll pull height weight and position then impute missing values that way

In [ ]:
url = 'http://stats.nba.com/stats/leaguedashplayerbiostats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=F&Season=2017-18&SeasonSegment=&SeasonType=Playoffs&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='
endpt,params = get_params(url)
get_nba_data(endpt,params).head()

Position is going to be slightly more complicated but we have a plan!

In [11]:
drafted_from_college = np.array(draft_history.index)
from_college_qual = np.intersect1d(qualified_player_list,drafted_from_college)

In [12]:
combine_player_list = np.array(full_combine_df.index)

players_to_find = pd.Series(np.setdiff1d(from_college_qual,combine_player_list))

In [13]:
players_to_find_df = draft_history.loc[players_to_find,:][['PLAYER_NAME','SEASON','TEAM_ID','TEAM_NAME']]

In [ ]:
endpt = 'commonteamroster'
all_players = get_nba_data('commonallplayers',{'LeagueID':'00','Season':'2017-18','IsOnlyCurrentSeason':'0'})
player_team_dict = all_players.set_index('PERSON_ID')['TEAM_ID']
new_player_combine_df = pd.DataFrame(columns=full_combine_df.columns)

for index,row in players_to_find_df.iterrows():
    season = str(row.SEASON)[:4]+"-"+str(row.SEASON+1)[2:]
    try:
        params = {'Season':season,'TeamID':str(row.TEAM_ID)}
        df_temp = get_nba_data(endpt,params)
        df_temp = df_temp.set_index('PLAYER_ID')
        height = df_temp.HEIGHT[int(index)].split("-")
        height_inches = int(height[0])*12 + int(height[1])-1
        weight = df_temp.WEIGHT[int(index)]
        position = df_temp.POSITION[int(index)]
#        print('trying')
    except:
#        print('except')
        try:
            team_id = player_team_dict[int(index)]
        except:
            continue
        params = {'Season':'2017-18','TeamID':str(team_id)}
        season_curr = 2017
        while(team_id==0 and season_curr>=2010):
            season_str = str(season_curr)[:4]+"-"+str(season_curr+1)[2:]
            all_players_temp = get_nba_data('commonallplayers',{'LeagueID':'00','Season':season_str,
                                                           'IsOnlyCurrentSeason':'1'})
            player_team_dict_temp = all_players_temp.set_index('PERSON_ID')['TEAM_ID']
            try:
                team_id = player_team_dict_temp[int(index)]
            except: 
                break
            params = {'Season':season_str,'TeamID':str(team_id)}
            season_curr-=1
#            print(season_curr)
#            print(season_str)
        try:
            df_temp = get_nba_data(endpt,params)
        except:
            continue
        df_temp = df_temp.set_index('PLAYER_ID')
        height = df_temp.HEIGHT[int(index)].split("-")
        height_inches = int(height[0])*12 + int(height[1])-1
        weight = df_temp.WEIGHT[int(index)]
        position = df_temp.POSITION[int(index)]
    new_row = pd.Series({'PLAYER_ID':index,
                         'PLAYER_NAME':row.PLAYER_NAME,
                         'DRAFT_YEAR':row.SEASON,
                         'HEIGHT_WO_SHOES':height_inches,
                         'WEIGHT':weight,'POSITION':position
                        }).transpose()
    new_player_combine_df=new_player_combine_df.append(new_row,ignore_index=True)
#    print([height,weight,position])
new_player_combine_df = new_player_combine_df.set_index('PLAYER_ID')

In [ ]:
new_player_combine_df

In [ ]:
full_combine_df = pd.concat([full_combine_df,new_player_combine_df])

In [ ]:
full_combine_df

In [ ]:
full_combine_df['POSITION_SIMPLE']=full_combine_df.POSITION
full_combine_df.POSITION_SIMPLE[full_combine_df.POSITION_SIMPLE.str.contains('C')]='Big'
full_combine_df.POSITION_SIMPLE[full_combine_df.POSITION_SIMPLE.str.contains('PF')]='Forward'
full_combine_df.POSITION_SIMPLE[full_combine_df.POSITION_SIMPLE.str.contains('SF')]='Wing'
full_combine_df.POSITION_SIMPLE[full_combine_df.POSITION_SIMPLE.str.contains('G')]='Guard'
full_combine_df.POSITION_SIMPLE[full_combine_df.POSITION_SIMPLE.str.contains('F')]='Forwad'

full_combine_df.loc[:,['POSITION','POSITION_SIMPLE']]

In [ ]:
full_combine_df = full_combine_df.apply(pd.to_numeric,errors='ignore')
full_combine_df = full_combine_df.loc[~full_combine_df.WEIGHT.isna(),:]
full_combine_df

In [ ]:
full_combine_df.groupby('POSITION_SIMPLE').describe().stack()

Now we begin to work on imputing. We're going to fit a predictive model for each feature based on height weight and positiion. Lets try lane agility time for starters.

In [ ]:
lane_agility_missing.groupby('POSITION_SIMPLE')['LANE_AGILITY_TIME'].describe()

Time to work on something that does this for the whole dataframe

In [ ]:
predicting_vals = pd.get_dummies(full_combine_df[['DRAFT_YEAR','POSITION_SIMPLE','WEIGHT','HEIGHT_WO_SHOES']])
predicting_vals
cols_to_impute = ['STANDING_VERTICAL_LEAP','MAX_VERTICAL_LEAP','LANE_AGILITY_TIME',
                  'THREE_QUARTER_SPRINT','HEIGHT_WO_SHOES','WINGSPAN','STANDING_REACH',
                  'BODY_FAT_PCT']
missing = full_combine_df[cols_to_impute[0]].isna()
X  = predicting_vals[~missing]
Y = full_combine_df[cols_to_impute[0]][~missing]
X_fill = predicting_vals[missing]

for col in cols_to_impute:
    missing = full_combine_df[[col]].isna()
    X = predicting_vals[~missing]
    Y = full_combine_df[col][~missing]
    X_fill = predicting_vals[missing]
    model = LinearRegression(fit_intercept=True)
    model.fit(X,Y)
    full_combine_df[col][missing]=model.predict(X_fill)


,PLAYER_NAME,DRAFT_YEAR,POSITION,STANDING_VERTICAL_LEAP,MAX_VERTICAL_LEAP,LANE_AGILITY_TIME,THREE_QUARTER_SPRINT,HEIGHT_WO_SHOES,WEIGHT,WINGSPAN,STANDING_REACH,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH,POSITION_SIMPLE
201570,Joe Alexander,2008,SF-PF,32.500000,38.5,11.33,2.99,79.25,220.0,83.50,106.0,5.8,NaN,NaN,Forwad
201589,Darrell Arthur,2008,PF,28.500000,30.0,12.18,3.14,79.50,216.0,82.75,107.0,7.3,NaN,NaN,Forwad
201571,D.J. Augustin,2008,PG,28.500000,35.0,11.27,3.07,70.00,171.5,75.50,94.5,8.1,NaN,NaN,Guard
201573,Jerryd Bayless,2008,PG-SG,31.000000,38.0,11.26,3.07,73.75,204.0,75.50,97.0,4.7,NaN,NaN,Guard
201563,Michael Beasley,2008,PF-SF,30.000000,35.0,11.06,3.24,79.00,239.0,84.25,107.0,7.7,NaN,NaN,Forwad
12239,Ramel Bradley,2008,PG-SG,24.500000,29.5,10.68,3.24,73.25,184.0,77.00,98.0,5.7,NaN,NaN,Guard
12240,Tyrone Brazelton,2008,PG,30.000000,34.0,11.59,3.25,70.00,166.0,73.75,94.0,8.1,NaN,NaN,Guard
12241,Takais Brown,2008,PF,29.000000,31.0,12.47,3.25,79.25,252.0,82.00,106.0,9.2,NaN,NaN,Forwad
201812,Keith Brumbaugh,2008,SF-PF,26.000000,27.0,11.65,3.28,79.50,190.0,79.25,104.5,4.0,NaN,NaN,Forwad
201868,Stanley Burrell,2008,PG,29.500000,34.5,11.19,3.19,73.50,201.0,78.25,97.5,5.6,NaN,NaN,Guard
